# [E-14] Recommendata Movie

### **활용할 데이터셋은 추천 시스템의 MNIST라고 부를만한 `Movielens` 데이터입니다.**

- **유저가 영화에 대해 평점을 매긴 데이터가 데이터 크기 별로 있습니다. `MovieLens 1M Dataset` 사용을 권장합니다.**
- **별점 데이터는 대표적인 explicit(명백한) 데이터입니다. 하지만 implicit(절대적) 데이터로 간주하고 테스트해 볼 수 있습니다.**
- **별점을 시청횟수로 해석**
- **또한 유저가 3점 미만으로 준 데이터는 선호하지 않는다고 가정하고 제외**

## Step 1 데이터 준비와 전처리

Movielens 데이터는 'rating.dat' 안에 이미 인덱싱까지 완료된 사용자-영화-평점 데이터가 깔끔하게 정리되어 있다.

1.1 데이터 가져오기

In [1]:
import os
import pandas as pd

In [2]:
# 데이터 가져와서 보기
rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'ratings', 'timestamp']
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python', encoding = "ISO-8859-1")
orginal_data_size = len(ratings)
ratings.head()

,user_id,movie_id,ratings,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [3]:
ratings.drop('timestamp', axis=1, inplace=True)
ratings.head()

,user_id,movie_id,ratings
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5


In [4]:
ratings.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000209 entries, 0 to 1000208
Data columns (total 3 columns):
 #   Column    Non-Null Count    Dtype
---  ------    --------------    -----
 0   user_id   1000209 non-null  int64
 1   movie_id  1000209 non-null  int64
 2   ratings   1000209 non-null  int64
dtypes: int64(3)
memory usage: 22.9 MB


In [5]:
# 평점 3점 이상만 남기기
ratings = ratings[ratings['ratings']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000209, filtered_data_size: 836478
Ratio of Remaining Data is 83.63%


In [6]:
# ratings 컬럼의 이름을 counts로 바꾸기
ratings.rename(columns={'ratings':'counts'}, inplace=True)
ratings.head()

,user_id,movie_id,counts
0,1,1193,5
1,1,661,3
2,1,914,3
3,1,3408,4
4,1,2355,5


1.2 영화 제목을 보기 위해 movies 데이터 가져와서 보기

In [7]:
# 영화 제목을 보기 위해 movies 데이터 가져와서 보기
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding='ISO-8859-1')
movies.head(10)

,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy
5,6,Heat (1995),Action|Crime|Thriller
6,7,Sabrina (1995),Comedy|Romance
7,8,Tom and Huck (1995),Adventure|Children's
8,9,Sudden Death (1995),Action
9,10,GoldenEye (1995),Action|Adventure|Thriller


In [8]:
movies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3883 entries, 0 to 3882
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   movie_id  3883 non-null   int64 
 1   title     3883 non-null   object
 2   genre     3883 non-null   object
dtypes: int64(1), object(2)
memory usage: 91.1+ KB


In [9]:
ratings['user_id'].nunique()

6039

## Step 2 분석하기

### 2.1 ratings, movies를 movid_id로 병합해서 통합 data를 만들기

- **data에 있는 유니크한 영화 개수**
- **data에 있는 유니크한 사용자 수**
- **가장 인기 있는 영화 30개(인기순)**

In [10]:
data = pd.merge(ratings, movies)
data.sample(10)

,user_id,movie_id,counts,title,genre
354734,1078,2125,4,Ever After: A Cinderella Story (1998),Drama|Romance
655387,2753,3730,4,"Conversation, The (1974)",Drama|Mystery
239592,2106,2890,4,Three Kings (1999),Drama|War
501806,198,3461,3,Lord of the Flies (1963),Adventure|Drama|Thriller
148572,534,2006,4,"Mask of Zorro, The (1998)",Action|Adventure|Romance
691610,1603,3060,5,"Commitments, The (1991)",Comedy|Drama
595571,869,2792,4,Airplane II: The Sequel (1982),Comedy
247661,1616,1732,4,"Big Lebowski, The (1998)",Comedy|Crime|Mystery|Thriller
554949,4399,3396,4,"Muppet Movie, The (1979)",Children's|Comedy
232881,1552,2599,5,Election (1999),Comedy


In [11]:
# 유저 수
print("원래 유저의 수 :", ratings['user_id'].nunique(), "\n병합 후 유저 수 :", data['user_id'].nunique())

원래 유저의 수 : 6039 
병합 후 유저 수 : 6039


In [12]:
# 영화수
print("원래 영화의 수 :", movies['movie_id'].nunique(), "\n병합 후 영화 수 :", data['movie_id'].nunique())

원래 영화의 수 : 3883 
병합 후 영화 수 : 3628


In [13]:
# 인기 많은 영화
title_count = data.groupby('title')['user_id'].count()
title_count.sort_values(ascending=False).head(30)

title
American Beauty (1999)                                   3211
Star Wars: Episode IV - A New Hope (1977)                2910
Star Wars: Episode V - The Empire Strikes Back (1980)    2885
Star Wars: Episode VI - Return of the Jedi (1983)        2716
Saving Private Ryan (1998)                               2561
Terminator 2: Judgment Day (1991)                        2509
Silence of the Lambs, The (1991)                         2498
Raiders of the Lost Ark (1981)                           2473
Back to the Future (1985)                                2460
Matrix, The (1999)                                       2434
Jurassic Park (1993)                                     2413
Sixth Sense, The (1999)                                  2385
Fargo (1996)                                             2371
Braveheart (1995)                                        2314
Men in Black (1997)                                      2297
Schindler's List (1993)                                  2257
Pr

In [14]:
# 유저별 몇 편의 영화를 보는지에 대한 통계
user_count = data.groupby('user_id')['title'].count()
user_count.describe()

count    6039.000000
mean      138.512668
std       156.241599
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: title, dtype: float64

## step 3 내가 선호하는 영화를 5가지 골라서 ratings에 추가하기
- 이 데이터셋은 너무 작아서 내가 좋아하는 영화 찾기가 힘들다.
- [movie dataset](https://github.com/databricks/spark-training/tree/master/data/movielens)
- 제목이 애매해서 movie_id를 추가  
|movie_id|title|
|:------|:------|
| 2571 | Matrix(1999) |
| 1805 | Wild Things(1998)|
| 280 | Murder in the First(1995)|
| 589 | Teminator2(1991)|
| 587 | Ghost(1990) |

### 3.1  좋아하는 영화 ratings에 movie_id로 추가

In [15]:
# 내가 좋아하는 영화 ratings 데이터에 추가하기(단,이름은 꼭 데이터셋에 있는 것을 넣어주기)
my_favorite = [2571, 1805, 280, 589 , 587]

# '6041'이라는 user_id가 위 영화의 평점을 5점씩 준것
my_playlist = pd.DataFrame({'user_id': [6041]*5, 'movie_id': my_favorite, 'counts':[5]*5})

if not ratings.isin({'user_id':[6041]})['user_id'].any():  # user_id에 'flower'이라는 데이터가 없다면
    ratings = ratings.append(my_playlist)                           # 위에 임의로 만든 my_favorite 데이터를 추가

ratings.tail(10)       # 잘 추가되었는지 확인

,user_id,movie_id,counts
1000203,6040,1090,3
1000205,6040,1094,5
1000206,6040,562,5
1000207,6040,1096,4
1000208,6040,1097,4
0,6041,2571,5
1,6041,1805,5
2,6041,280,5
3,6041,589,5
4,6041,587,5


### 3.2 ratings, movies를 movid_id로 병합해서 통합 data를 만들기

In [16]:
data = pd.merge(ratings, movies)
data

,user_id,movie_id,counts,title,genre
0,1,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama
1,2,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama
2,12,1193,4,One Flew Over the Cuckoo's Nest (1975),Drama
3,15,1193,4,One Flew Over the Cuckoo's Nest (1975),Drama
4,17,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama
...,...,...,...,...,...
836478,5851,3607,5,One Little Indian (1973),Comedy|Drama|Western
836479,5854,3026,4,Slaughterhouse (1987),Horror
836480,5854,690,3,"Promise, The (Versprechen, Das) (1994)",Romance
836481,5938,2909,4,"Five Wives, Three Secretaries and Me (1998)",Documentary


**추가해 준 데이터가 잘 들어갔는 지 확인**

In [17]:
print(data[data.user_id == 6041])

        user_id  movie_id  counts                              title  \
91924      6041       589       5  Terminator 2: Judgment Day (1991)   
124654     6041      2571       5                 Matrix, The (1999)   
361469     6041       587       5                       Ghost (1990)   
516967     6041      1805       5                 Wild Things (1998)   
724973     6041       280       5         Murder in the First (1995)   

                               genre  
91924         Action|Sci-Fi|Thriller  
124654        Action|Sci-Fi|Thriller  
361469       Comedy|Romance|Thriller  
516967  Crime|Drama|Mystery|Thriller  
724973                Drama|Thriller  


In [18]:
data.sort_values(by="user_id", axis=0, inplace=True)
data

,user_id,movie_id,counts,title,genre
0,1,1193,5,One Flew Over the Cuckoo's Nest (1975),Drama
31113,1,2294,4,Antz (1998),Animation|Children's
31674,1,3186,4,"Girl, Interrupted (1999)",Drama
32044,1,1566,4,Hercules (1997),Adventure|Animation|Children's|Comedy|Musical
32415,1,588,4,Aladdin (1992),Animation|Children's|Comedy|Musical
...,...,...,...,...,...
724973,6041,280,5,Murder in the First (1995),Drama|Thriller
361469,6041,587,5,Ghost (1990),Comedy|Romance|Thriller
124654,6041,2571,5,"Matrix, The (1999)",Action|Sci-Fi|Thriller
91924,6041,589,5,Terminator 2: Judgment Day (1991),Action|Sci-Fi|Thriller


In [19]:
# 고유한 유저, 영화를 찾아내는 코드
user_unique = data['user_id'].unique()
movie_unique = data['movie_id'].unique()


# {유저 id:인덱스번호}로 구성된 딕셔너리 반환
user_to_idx = {v:k for k,v in enumerate(user_unique)}

# {movie_id:인덱스번호}로 구성된 딕셔너리 반환
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [20]:
# 인덱싱 확인
print(user_to_idx[6041])    
print(movie_to_idx[2571])

6039
76


In [21]:
# indexing을 통해 데이터 컬럼 내 값을 바꾸는 코드
# dictionary 자료형의 get 함수는 https://wikidocs.net/16 을 참고하세요.

# user_to_idx.get을 통해 user_id 컬럼의 모든 값을 인덱싱한 Series를 구해 봅시다. 
# 혹시 정상적으로 인덱싱되지 않은 row가 있다면 인덱스가 NaN이 될 테니 dropna()로 제거합니다. 
temp_user_data = data['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(data):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    data['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = data['movie_id'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(data):
    print('movie_id column indexing OK!!')
    data['movie_id'] = temp_movie_data
else:
    print('movie_id column indexing Fail!!')

data

user_id column indexing OK!!
movie_id column indexing OK!!


,user_id,movie_id,counts,title,genre
0,0,0,5,One Flew Over the Cuckoo's Nest (1975),Drama
31113,0,1,4,Antz (1998),Animation|Children's
31674,0,2,4,"Girl, Interrupted (1999)",Drama
32044,0,3,4,Hercules (1997),Adventure|Animation|Children's|Comedy|Musical
32415,0,4,4,Aladdin (1992),Animation|Children's|Comedy|Musical
...,...,...,...,...,...
724973,6039,1632,5,Murder in the First (1995),Drama|Thriller
361469,6039,659,5,Ghost (1990),Comedy|Romance|Thriller
124654,6039,76,5,"Matrix, The (1999)",Action|Sci-Fi|Thriller
91924,6039,156,5,Terminator 2: Judgment Day (1991),Action|Sci-Fi|Thriller


## Step 4. CSR matrix 만들기

In [22]:
# matrix 만들기
from scipy.sparse import csr_matrix

num_user = data['user_id'].nunique()
num_movie = data['movie_id'].nunique()

csr_data = csr_matrix((data.counts, (data.user_id, data.movie_id)), shape= (num_user, num_movie))
csr_data

<6040x3628 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Row format>

## **Step 5. MF 모델 학습하기**

**Matrix Factorization 모델을 [implicit](https://github.com/benfred/implicit) 패키지를 사용하여 학습**

- **`implicit` 패키지는 이전 스텝에서 설명한 암묵적(implicit) dataset을 사용하는 다양한 모델을 굉장히 빠르게 학습할 수 있는 패키지**
- **이 패키지에 구현된 `als(AlternatingLeastSquares) 모델`을 사용**
    - **한쪽을 고정시키고 다른 쪽을 학습하는 방식을 번갈아 수행하는 방식**

**implicit 패키지 불러오기, 세팅**

In [23]:
# implicit 패키지 불러오기, 세팅
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하고 있는 부분입니다. 학습 내용과는 무관합니다.
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

**AlternatingLeastSquares 클래스의 __init__ 파라미터**

1. **factors : 유저와 아이템의 벡터를 몇 차원으로 할 것인지**
2. **regularization : 과적합을 방지하기 위해 정규화 값을 얼마나 사용할 것인지**
3. **use_gpu : GPU를 사용할 것인지**
4. **iterations : epochs와 같은 의미(데이터를 몇 번 반복해서 학습할 것인지)**

**`1과 4를 늘릴수록` 학습 데이터를 잘 학습하게 되지만 `과적합`의 우려가 있으니 좋은 값을 찾기**

In [24]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=200, regularization=0.01, use_gpu=False, iterations=20, dtype=np.float32)

In [25]:
# als 모델은 input으로 (item X user)꼴의 matrix를 받기 때문에 Transpose
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [26]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/20 [00:00<?, ?it/s]

**als_model을 이용해서 flower_vector, matrix_vector 만들기**

In [27]:
# als_model을 이용해서 flower_vector, matrix_vector 만들기
flower, matrix = user_to_idx[6041], movie_to_idx[2571]
flower_vector, matrix_vector = als_model.user_factors[flower], als_model.item_factors[matrix]

In [28]:
# flower_vector 확인
flower_vector.shape

(200,)

In [29]:
# matrix_vector 확인
matrix_vector.shape

(200,)

**flower와 matrix 선호도 예측**

In [30]:
# flower과 matrix 내적
np.dot(flower_vector, matrix_vector)

0.7259715

**flower, terminator 선호도 예측**

In [31]:
# terminator에 대한 flower의 선호도를 예측 보기

terminator = movie_to_idx[1240]
terminator_vector = als_model.item_factors[terminator]
np.dot(flower_vector, terminator_vector)

0.3735022

## Step 7. 내가 좋아하는 영화와 비슷한 영화를 추천받기

In [32]:
def similar_movie_top15(movie_id, N):
    # movie_id와 비슷한 영화 찾기
    favorite_movie = movie_id
    movie_num = movie_to_idx[favorite_movie]
    similar_movie = als_model.similar_items(movie_num, N=N)
    
    # movie_id에서 영화제목 추출하는 함수
    def get_title_from_index(index):
        return data[data.movie_id == index]["title"].values[0]
    
    recommen_movie = get_title_from_index(movie_num)
    
    # 영화제목, movie_id, 유사도 저장
    similar_movie_final = []

    print(recommen_movie + " 비슷한 영화 15편 \n")
    for item in similar_movie:
        similar_movie_final.append((get_title_from_index(item[0]), item[0], item[1]))

    for i in similar_movie_final:
        print(i)    

In [33]:
similar_movie_top15(587,15)

Ghost (1990) 비슷한 영화 15편 

('Ghost (1990)', 659, 1.0000001)
('Pretty Woman (1990)', 334, 0.61136836)
('Sleepless in Seattle (1993)', 601, 0.44220936)
('Mrs. Doubtfire (1993)', 825, 0.42493403)
('Flatliners (1990)', 1197, 0.4188093)
('While You Were Sleeping (1995)', 593, 0.41804793)
("My Best Friend's Wedding (1997)", 353, 0.40454543)
("You've Got Mail (1998)", 708, 0.3913477)
('Ransom (1996)', 836, 0.38551295)
("Mummy's Curse, The (1944)", 2155, 0.37529215)
('Shattered Image (1998)', 2957, 0.3748817)
('Stranger in the House (1997)', 3560, 0.3691638)
('Rough Night in Jericho (1967)', 3112, 0.3682615)
("I Love You, Don't Touch Me! (1998)", 3366, 0.3667698)
('French Kiss (1995)', 1255, 0.36608097)


In [34]:
similar_movie_top15(589,15)

Terminator 2: Judgment Day (1991) 비슷한 영화 15편 

('Terminator 2: Judgment Day (1991)', 156, 1.0000001)
('Terminator, The (1984)', 198, 0.6088203)
('Matrix, The (1999)', 76, 0.60242194)
('Total Recall (1990)', 145, 0.542213)
('Jurassic Park (1993)', 138, 0.47044718)
('Men in Black (1997)', 173, 0.38637596)
('Fugitive, The (1993)', 100, 0.36018556)
('Aliens (1986)', 721, 0.33549538)
("She's the One (1996)", 1237, 0.3141501)
('Committed (2000)', 2736, 0.3080015)
('Hunt for Red October, The (1990)', 119, 0.3068021)
('Germinal (1993)', 3296, 0.30315712)
('Bittersweet Motel (2000)', 3510, 0.30287078)
('Braveheart (1995)', 107, 0.30243626)
('Apple, The (Sib) (1998)', 3308, 0.30221507)


## Step 8. 내가 좋아할 만한 영화 추천받기

In [35]:
def recommend_movie(user_id, N):
    # flower 좋아하는 영화 추천 인덱스 추출
    user = user_to_idx[user_id]
    # recommend에서는 user*item CSR Matrix를 받습니다.
    movie_recommend= als_model.recommend(user, csr_data, N=N, filter_already_liked_items=True)
    
    # movie_id에서 영화제목 추출하는 함수
    def get_title_from_index(index):
        return data[data.movie_id == index]["title"].values[0]
    
    
    # 영화제목, movie_id, 유사도 저장
    movie_recommend_final = []

    print(f"추천영화 {N}편 \n")
    for item in movie_recommend:
        movie_recommend_final.append((get_title_from_index(item[0]), item[0], item[1]))

    for i in movie_recommend_final:
        print(i)

In [36]:
recommend_movie(6041, 20)

추천영화 20편 

('Terminator, The (1984)', 198, 0.3735022)
('Total Recall (1990)', 145, 0.31615427)
('Ransom (1996)', 836, 0.26637346)
('Men in Black (1997)', 173, 0.25136378)
('Jurassic Park (1993)', 138, 0.22399093)
('Bound (1996)', 304, 0.22397742)
('Seven (Se7en) (1995)', 314, 0.21138716)
("Devil's Advocate, The (1997)", 792, 0.209808)
('Face/Off (1997)', 373, 0.20440173)
('Fugitive, The (1993)', 100, 0.20046632)
('Primal Fear (1996)', 824, 0.19850679)
('Saving Private Ryan (1998)', 19, 0.1810452)
('Usual Suspects, The (1995)', 267, 0.17583475)
('Firm, The (1993)', 409, 0.17279582)
('Kiss the Girls (1997)', 1517, 0.16778824)
('Spanish Prisoner, The (1997)', 157, 0.16707858)
('Forrest Gump (1994)', 131, 0.16694039)
('Braveheart (1995)', 107, 0.16682523)
('Game, The (1997)', 797, 0.16630489)
('Titanic (1997)', 25, 0.16529885)


## Step 8. factors=500 <- 처음에 해 본 것.. 개인적으로 위의 결과가 맘에 들어 위의 모델 선택

- 벡터의 크기를 크게 했더니, 내가 추천한 영화와의 선호도는 매우 좋다
- 하지만, 내가 선택하지 않은 영화와의 선호도는 별로 좋지 못하다.
- 과적합인 것 같다.

In [24]:
# Implicit AlternatingLeastSquares 모델의 선언
als_model = AlternatingLeastSquares(factors=500, regularization=0.01, use_gpu=False, iterations=20, dtype=np.float32)

In [25]:
# als 모델은 input으로 (item X user)꼴의 matrix를 받기 때문에 Transpose
csr_data_transpose = csr_data.T
csr_data_transpose

<3628x6040 sparse matrix of type '<class 'numpy.int64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [26]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/20 [00:00<?, ?it/s]

**als_model을 이용해서 flower_vector, matrix_vector 만들기**

In [27]:
# als_model을 이용해서 flower_vector, matrix_vector 만들기
flower, matrix = user_to_idx[6041], movie_to_idx[2571]
flower_vector, matrix_vector = als_model.user_factors[flower], als_model.item_factors[matrix]

In [28]:
# flower_vector 확인
flower_vector.shape

(500,)

In [29]:
# matrix_vector 확인
matrix_vector.shape

(500,)

**flower와 matrix 선호도 예측**

In [30]:
# flower과 matrix 내적
np.dot(flower_vector, matrix_vector)

0.9452948

**flower, terminator에 선호도 예측**

In [31]:
# terminator에 대한 flower의 선호도를 예측 보기

terminator = movie_to_idx[1240]
terminator_vector = als_model.item_factors[terminator]
np.dot(flower_vector, terminator_vector)

0.22865026

**내가 좋아하는 영화와 비슷한 영화를 추천받기**

In [32]:
def similar_movie_top15(movie_id, N):
    # movie_id와 비슷한 영화 찾기
    favorite_movie = movie_id
    movie_num = movie_to_idx[favorite_movie]
    similar_movie = als_model.similar_items(movie_num, N=N)
    
    # movie_id에서 영화제목 추출하는 함수
    def get_title_from_index(index):
        return data[data.movie_id == index]["title"].values[0]
    
    recommen_movie = get_title_from_index(movie_num)
    
    # 영화제목, movie_id, 유사도 저장
    similar_movie_final = []

    print(recommen_movie + " 비슷한 영화 15편 \n")
    for item in similar_movie:
        similar_movie_final.append((get_title_from_index(item[0]), item[0], item[1]))

    for i in similar_movie_final:
        print(i)    

In [33]:
similar_movie_top15(587,15)

Ghost (1990) 비슷한 영화 15편 

('Ghost (1990)', 659, 1.0000001)
('Shattered Image (1998)', 2957, 0.4041616)
('Mr. Magoo (1997)', 2347, 0.39524364)
('Gay Deceivers, The (1969)', 3569, 0.39139843)
('Held Up (2000)', 3493, 0.39015025)
('Molly (1999)', 3158, 0.3880006)
('Rent-a-Kid (1995)', 3193, 0.3875163)
('Leather Jacket Love Story (1997)', 3506, 0.3873704)
('In the Line of Duty 2 (1987)', 3353, 0.3871596)
('Stranger, The (1994)', 3605, 0.38701534)
('Rough Night in Jericho (1967)', 3112, 0.38671765)
('Zachariah (1971)', 3559, 0.38657045)
('Second Best (1994)', 3583, 0.38585648)
('Brother Minister: The Assassination of Malcolm X (1994)', 3607, 0.38561806)
('Stranger in the House (1997)', 3560, 0.38553244)


In [34]:
similar_movie_top15(589,15)

Terminator 2: Judgment Day (1991) 비슷한 영화 15편 

('Terminator 2: Judgment Day (1991)', 156, 1.0000001)
('Grosse Fatigue (1994)', 3268, 0.37656322)
('Man from Down Under, The (1943)', 3481, 0.37334514)
('City of the Living Dead (Paura nella città dei morti viventi) (1980)', 3512, 0.37313)
('Sorority House Massacre II (1990)', 3183, 0.37274668)
('Schlafes Bruder (Brother of Sleep) (1995)', 3466, 0.36939606)
("I Can't Sleep (J'ai pas sommeil) (1994)", 3281, 0.3693751)
('Running Free (2000)', 3156, 0.3686136)
('Stranger, The (1994)', 3605, 0.3679255)
('Ulysses (Ulisse) (1954)', 3544, 0.36763322)
('Simon Sez (1999)', 3499, 0.36685023)
('Institute Benjamenta, or This Dream People Call Human Life (1995)', 3303, 0.3664202)
('Bittersweet Motel (2000)', 3510, 0.36613166)
('Getting Away With Murder (1996)', 3173, 0.36601803)
("Billy's Holiday (1995)", 2986, 0.3657377)


## Step 8. 내가 좋아할 만한 영화 추천받기

In [35]:
def recommend_movie(user_id, N):
    # flower 좋아하는 영화 추천 인덱스 추출
    user = user_to_idx[user_id]
    # recommend에서는 user*item CSR Matrix를 받습니다.
    movie_recommend= als_model.recommend(user, csr_data, N=N, filter_already_liked_items=True)
    
    # movie_id에서 영화제목 추출하는 함수
    def get_title_from_index(index):
        return data[data.movie_id == index]["title"].values[0]
    
    
    # 영화제목, movie_id, 유사도 저장
    movie_recommend_final = []

    print(f"추천영화 {N}편 \n")
    for item in movie_recommend:
        movie_recommend_final.append((get_title_from_index(item[0]), item[0], item[1]))

    for i in movie_recommend_final:
        print(i)

In [36]:
recommend_movie(6041, 20)

추천영화 20편 

('Terminator, The (1984)', 198, 0.22865026)
('Primal Fear (1996)', 824, 0.17956488)
('Ransom (1996)', 836, 0.17051807)
('Far and Away (1992)', 432, 0.11836855)
("Devil's Advocate, The (1997)", 792, 0.11472375)
('Tao of Steve, The (2000)', 1471, 0.11434703)
('Seven (Se7en) (1995)', 314, 0.11287715)
('Mask, The (1994)', 470, 0.11200081)
('Kiss the Girls (1997)', 1517, 0.1115242)
('Firm, The (1993)', 409, 0.11137301)
('Sleepers (1996)', 1385, 0.1093062)
('Gattaca (1997)', 383, 0.1087544)
('Lost World: Jurassic Park, The (1997)', 112, 0.10836181)
('Bound (1996)', 304, 0.10717656)
('Men in Black (1997)', 173, 0.10671297)
('Man for All Seasons, A (1966)', 1452, 0.10461156)
('Spanish Prisoner, The (1997)', 157, 0.10371525)
('Total Recall (1990)', 145, 0.10240842)
('Graduate, The (1967)', 158, 0.10131778)
('North by Northwest (1959)', 244, 0.10114087)


## 회고

1. 이번 프로젝트에서 **어려웠던 점**
- 유사도를 측정하기 위해 'title'을 이용했어야 하나, 'title'의 'the'가 뒤에 있는 것이 있어서 movie_id를 이용해서 추천 받음
- 추천 영화 제목과 movie_id를 조인하는 것이 어려웠는 데, fundamentals에 있는 함수 사용함
2. 프로젝트를 진행하면서 **알아낸 점**
- csr Matrix를 만드는 방법
3. 프로젝트를 진행하면서 **아직 모호한 점**
- 'title'에서 'the'의 위치가 뒤에 있는 것을 전처리 하고 프로젝트를 했어야 하나??
    - '그렇다면 어디까지 전처리를 해야 하는가?', 시리즈물은 어떻게 하지?
- 추천 시스템을 만들려면 정확한 'title'을 줘야 하는 데, 그건 서비스해야 할 때 신경써야 하는 부분인 것 같은 데..
4. **자기 다짐**
- exploration을 진행하다보니 이제는 판다스도 해야할 것 같다..
- 할 것은 쌓여 가는 데 우짜지?? ㅎㅎ